In [1]:
import nltk.classify
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.tokenize import word_tokenize
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from collections import defaultdict
from nltk.metrics import precision, recall

In [1]:
import spacy

# We only need the sentence splitting and word embedding features, disabling the
# unneed features makes this a lot faster.
nlp = spacy.load('en_core_web_md', disable=['tagger', 'ner'])

'''
TODO: word embeddings aan de dataset toevoegen
Denk dat bij het inlezen dit het beste is om te doen, dan hoeven we maar 1x door alles
heen te loopen. Even uitzoeken hoe de embedding vectors meegegeven moeten worden 
(dict net als BOW of tuple met (<lijst met vectors>, <label>))

Even experimenteren met wat beter werkt: 
    - soort bow van individuele token vectors per review
    - of een gecombineerde methode gebruiken 
        -> gemiddelde van alle token vectors
        -> doc vector
        -> ???

'''

'\nTODO: word embeddings aan de dataset toevoegen\nDenk dat bij het inlezen dit het beste is om te doen, dan hoeven we maar 1x door alles\nheen te loopen. Even uitzoeken hoe de embedding vectors meegegeven moeten worden \n(dict net als BOW of tuple met (<lijst met vectors>, <label>))\n\nEven experimenteren met wat beter werkt: \n    - soort bow van individuele token vectors per review\n    - of een gecombineerde methode gebruiken \n        -> gemiddelde van alle token vectors\n        -> doc vector\n        -> ???\n\n'

In [13]:
nlp('i.e').vector

array([-2.5118e-01, -6.0606e-02, -3.4313e-01,  3.7411e-02, -8.4498e-02,
        1.2884e-02,  2.4850e-01, -1.9731e-02,  2.4863e-01,  1.0369e+00,
       -2.2219e-01,  4.5068e-01,  1.8731e-02,  3.5804e-01,  2.8142e-01,
        1.0846e-01, -2.8425e-01,  1.4556e+00,  1.3171e-01,  1.2139e-01,
       -3.7228e-01,  1.6142e-01, -7.8251e-02, -3.4224e-02,  2.8212e-01,
       -3.0390e-01, -7.7036e-02, -6.9127e-02,  2.8540e-01,  2.1351e-01,
       -9.2228e-03,  1.2881e-01,  1.9785e-01,  3.0749e-01, -1.4267e-02,
        1.8628e-01,  7.1090e-03,  2.7434e-01,  1.1826e-01,  2.5580e-02,
       -1.7196e-01,  9.0421e-02, -3.5220e-01,  3.8906e-01, -2.4534e-01,
        8.6821e-02,  2.1425e-01, -2.3036e-02,  9.6637e-02,  1.3713e-01,
        2.2984e-01,  9.2558e-03,  2.1766e-01, -3.5353e-01, -8.6038e-02,
       -3.1343e-01, -2.0462e-01, -4.3504e-01, -1.6242e-01,  1.3472e-01,
       -8.2011e-02,  2.6381e-02,  4.2198e-01, -2.8300e-01,  2.1186e-01,
       -3.4468e-01,  1.7031e-01, -9.3016e-02, -1.1092e-01, -2.45

In [7]:
import csv
import random
import sys
from string import punctuation
import pickle

from nltk import download, word_tokenize
from nltk.corpus import stopwords

# Download the nltk stopwords corpus if needed
download('stopwords')


def create_dataset(filepath):
    ''' Reads and cleans the csv file and structures the datapoints ''' 
    dataset = []

    # The translation and set are both a lot faster O(1) 
    # when compared to checking a list or string O(n).
    punct_translation = str.maketrans('', '', punctuation)
    stoplist = set(stopwords.words('english'))

    with open(filepath, 'r', encoding='latin-1') as f:
        reader = csv.reader(f, delimiter=",", )
        
        # Skip the header row
        next(reader, None)

        # Items per row:
        #   0 -> review id
        #   1 -> rating between 1-5
        #   2 -> year and month
        #   3 -> location of reviewer
        #   4 -> review text
        #   5 -> Disneyland location
        for row in reader:
            rating = int(row[1])

            if rating < 3:
                rating_label = 'negative'
            elif rating == 3:
                rating_label = 'indifferent'
            else:
                rating_label = 'positive'

            review_text = row[4] \
                .translate(punct_translation) \
                .lower() \
                .strip() + f"{row[5].replace('Disneyland_', '')} {row[3]}"

            tokenized = [
                token for token in word_tokenize(review_text)
                if token not in stoplist
            ]

            bag_of_words = ({t: True for t in tokenized}, rating)

            dataset.append(
                {
                    'tokenized': tokenized, 
                    'bag_of_words': bag_of_words, 
                    'rating_label': rating, 
                    'year_month': row[2], 
                    'reviewer_location': row[3], 
                    'review_text': row[4], 
                    'disneyland_location': row[5],
                    'doc_vector': nlp(' '.join(tokenized)).vector
                }
            )


    return dataset


def split_train_test(feats, split=0.8):
    ''' Creates test, train and dev splits from the dataset ''' 
    random.Random(1).shuffle(feats)

    cutoff = int(len(feats) * split)
    tenpercent = int((len(feats) - cutoff) / 2)
    split = cutoff + tenpercent

    train_feats = feats[:cutoff]
    test_feats = feats[cutoff:split]
    dev_feats = feats[split:]

    print("  Training set: %i" % len(train_feats))
    print("  Test set: %i" % len(test_feats))
    print("  Development set: %i" % len(dev_feats))

    return train_feats, test_feats, dev_feats
dataset = create_dataset('../data/DisneylandReviews.csv')

train_feats, test_feats, dev_feats = split_train_test(dataset)

[nltk_data] Downloading package stopwords to /home/wessel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


KeyboardInterrupt: 

In [141]:
def precision_recall(classifier, testfeats):
	refsets = defaultdict(set)
	testsets = defaultdict(set)
	
	for i, (feats, label) in enumerate(testfeats):
		refsets[label].add(i)
		observed = classifier.classify(feats)
		testsets[observed].add(i)
	
	precisions = {}
	recalls = {}
	
	for label in classifier.labels():
		precisions[label] = precision(refsets[label], testsets[label])
		recalls[label] = recall(refsets[label], testsets[label])
	
	return precisions, recalls

In [142]:
def calculate_f(precisions, recalls):
    f_measures = {}

    for category in precisions.keys():
        # This is done to prevent the program from crashing when 
        # no measure is provided for a particular category
        if not precisions[category] or not recalls[category]:
            f_measures[category] = None
            continue

        f_measures[category] = round(
            2 * ((precisions[category] * recalls[category]) /
                 (precisions[category] + recalls[category])), 6)

    return f_measures

In [143]:
def evaluation(classifier, test_feats, categories):
    """ Taken from assignment 1, calculates and prints evaluation measures """
    print("\n##### Evaluation...")
    print("  Accuracy: %f" % nltk.classify.accuracy(classifier, test_feats))
    precisions, recalls = precision_recall(classifier, test_feats)
    f_measures = calculate_f(precisions, recalls)

    print(" |-----------|-----------|-----------|-----------|")
    print(" |%-11s|%-11s|%-11s|%-11s|" %
          ("category", "precision", "recall", "F-measure"))
    print(" |-----------|-----------|-----------|-----------|")
    for category in categories:
        if precisions[category] is None:
            print(" |%-11s|%-11s|%-11s|%-11s|" % (category, "NA", "NA", "NA"))
        else:
            print(" |%-11s|%-11f|%-11f|%-11s|" %
                  (category,
                   precisions[category],
                   recalls[category],
                   f_measures[category])
                  )
    print(" |-----------|-----------|-----------|-----------|")

In [ ]:
def train_svm(train_feats):
    ''' Trains and returns a linear SVM classifier '''
    return SklearnClassifier(LinearSVC(dual=False)).train(train_feats)

def train_knn(train_feats):
    ''' Trains and returns a KNN classifier '''
    return SklearnClassifier(KNeighborsClassifier()).train(train_feats)

In [ ]:
only_bow_test = [item['bag_of_words'] for item in test_feats]
only_bow_train = [item['bag_of_words'] for item in train_feats]

In [ ]:
svm_classifier = train_svm(only_bow_train)

In [ ]:
knn_classifier = train_knn(only_bow_train)

In [ ]:
evaluation(svm_classifier, only_bow_test, ['positive', 'indifferent', 'negative'])
evaluation(knn_classifier, only_bow_test, ['positive', 'indifferent', 'negative'])

In [182]:
only_we_test = [(item['doc_vector'], item['rating_label']) for item in test_feats]
only_we_train = [(item['doc_vector'], item['rating_label']) for item in train_feats]

only_vec_train = [i[0] for i in only_we_train]
only_label_train = [j[1] for j in only_we_train]

only_vec_test = [i[0] for i in only_we_test]
only_label_test = [j[1] for j in only_we_test]

In [183]:
svm_classifier = LinearSVC().fit(only_vec_train, only_label_train)

In [184]:
svm_classifier.score(only_vec_test, only_label_test)

0.5977496483825597

In [154]:
from sklearn.tree import DecisionTreeClassifier

a = DecisionTreeClassifier().fit(only_vec_train, only_label_train)
a.score(only_vec_test, only_label_test)

0.43694327238631037

In [175]:
countries = {i['reviewer_location'] for i in dataset}
disneylands = {i['disneyland_location'].replace('Disneyland_', '') for i in dataset}

disneyland_lookup = { disneyland: nlp(disneyland).vector for disneyland in disneylands }
country_lookup = { country: nlp(country).vector for country in countries }

In [176]:
combined_features_train = []
labels_train = []

for row in train_feats:
    # We know these are always present in de dataset
    features = row['doc_vector']

    # if row['year_month'] == 'missing':
    #     features = np.concatenate((features, np.array([0, 0])))
    # else:
    #     # example format: 2019-4
    #     year, month = row['year_month'].split('-')
    #     features = np.concatenate((features, np.array([int(year), int(month)])))
    features = np.append(
        features, 
        disneyland_lookup[row['disneyland_location'].replace('Disneyland_', '')]
    )
    features = np.append(features, country_lookup[row['reviewer_location']])

    combined_features_train.append(features)
    labels_train.append(row['rating_label'])

In [177]:
combined_features_test = []
labels_test = []
for row in test_feats:
    # We know these are always present in de dataset
    features = row['doc_vector']
    # features = np.array([])

    # The feature arrays need to have the same shape, that is why we
    # insert 0s in here
    # if row['year_month'] == 'missing':
    #     features = np.concatenate((features, np.array([0, 0])))
    # else:
    #     # example format: 2019-4
    #     year, month = row['year_month'].split('-')
    #     features = np.concatenate((features, np.array([int(year), int(month)])))

    features = np.append(
        features, 
        disneyland_lookup[row['disneyland_location'].replace('Disneyland_', '')]
    )
    features = np.append(features, country_lookup[row['reviewer_location']])

    combined_features_test.append(features)
    labels_test.append(row['rating_label'])

In [178]:
svm_extra_features = LinearSVC().fit(combined_features_train, labels_train)
svm_extra_features.score(combined_features_test, labels_test)

/home/wessel/miniconda3/envs/mlp/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.5925925925925926

[1. 1.]


In [118]:
ex = dev_feats[9]

print(ex)

f = np.array([disneyland_lookup[ex['disneyland_location']], country_lookup[ex['reviewer_location']]])

print(f)

svm_extra_features.predict(f.reshape(1,-1))

{'tokenized': ['convenience', 'staffs', 'friendly', 'many', 'rude', 'mainland', 'ppl', 'didnt', 'need', 'line', 'long', 'time', 'rides', 'several', 'restaurants', 'provide', 'nice', 'food'], 'bag_of_words': ({'convenience': True, 'staffs': True, 'friendly': True, 'many': True, 'rude': True, 'mainland': True, 'ppl': True, 'didnt': True, 'need': True, 'line': True, 'long': True, 'time': True, 'rides': True, 'several': True, 'restaurants': True, 'provide': True, 'nice': True, 'food': True}, 'positive'), 'rating_label': 'positive', 'year_month': '2016-12', 'reviewer_location': 'Hong Kong', 'review_text': 'Convenience, Staffs are friendly ,, there are not many rude mainland ppl,, didnt need to line up for long time for rides,   There are several restaurants provide nice food.', 'disneyland_location': 'Disneyland_HongKong', 'doc_vector': array([ 8.26215744e-03,  2.48817932e-02, -1.40969366e-01, -2.98998266e-01,
        2.03529805e-01,  1.41831204e-01,  7.97833502e-02, -1.58968836e-01,
      

array(['positive'], dtype='<U11')

# Acc met onbewerkte, ruwe review text als doc vector (spacy schoont en splitst)
- SVM: 0.8218471636193155
- KNN: 0.7740271917487107

# Acc met tokenized en geschoonde tokens als doc vector 
- SVM: 0.8211439287388654
- KNN: 0.7805907172995781

# TODO
- Uitproberen met individuele token vectors en niet de 'platgeslagen' doc vector.
- Andere items uit de dataset als features toevoegen.

In [155]:
knn_classifier = KNeighborsClassifier().fit(only_vec_train, only_label_train)
knn_classifier.score(only_vec_test, only_label_test)

0.4997655883731833

In [16]:
import numpy as np

def cosine_similarity(a, b):
    return a.dot(b)/np.sqrt(a.dot(a) * b.dot(b))

In [17]:
from sklearn.metrics import precision_score, recall_score

def sklearn_model_evaluation(model, x_test, y_test):
    # y_pred = model.predict(x_test)
    # precision = precision_score(y_test, y_pred, average='micro')
    # recall = recall_score(y_test, y_pred, average='micro')
    accuracy = model.score(x_test, y_test)

    print("\n##### Evaluation...")
    print(f"  Accuracy: {accuracy}") 
    print(f"Precision: {precision}\nRecall: {recall}\nF-score: ")


In [18]:
print(prec svm_classifier, only_we_train)
print(prec knn_classifier, only_we_train)

SyntaxError: invalid syntax (<ipython-input-18-cc24e98a8505>, line 1)

In [67]:
sklearn_model_evaluation(svm_classifier, only_vec_test, only_label_test)
sklearn_model_evaluation(knn_classifier, only_vec_test, only_label_test)


##### Evaluation...
  Accuracy: 0.8211439287388654
Precision: 0.8211439287388654
Recall: 0.8211439287388654
F-score: 

##### Evaluation...
  Accuracy: 0.7805907172995781
Precision: 0.7805907172995781
Recall: 0.7805907172995781
F-score: 


In [ ]:
individuel token vectors als feature toevoegen
extra punten uit dataset toevoegen, kijken of daar regression of correlation mee gedaan kan worden

In [10]:
sorted([item['year_month'] for item in dataset if item['year_month'] != 'missing'])[-1]

'2019-5'

In [159]:
from sklearn.dummy import DummyClassifier

baseline = DummyClassifier(strategy="most_frequent")
baseline.fit(only_vec_train, only_label_train)
baseline.score(only_vec_test, only_label_test)

0.540084388185654

In [156]:
from sklearn.metrics import accuracy_score, confusion_matrix

confusion_matrix(labels_test, svm_classifier.predict(only_vec_test))

ValueError: Mix of label input types (string and number)

In [33]:
import csv
import os
import pickle
import random
import sys
from collections import defaultdict
from string import punctuation

import numpy as np
import spacy
from nltk import download, word_tokenize
from nltk.classify import accuracy
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.corpus import stopwords
from nltk.metrics import precision, recall
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.dummy import DummyClassifier
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             precision_score, recall_score)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

def create_dataset(filepath, use_cache=False):
    ''' Reads and cleans the csv file and structures the data '''

    print('Reading in files...')

    # Return early when a cache exists
    if use_cache and os.path.isfile(CACHE_PATH):
        with open(CACHE_PATH, 'rb') as cache:
            print('Used cached dataset!')
            return pickle.load(cache)

    nlp = spacy.load('en_core_web_md', disable=['tagger', 'ner'])

    dataset = []

    # The translation and set are both a lot faster O(1) when compared to
    # checking a list or string O(n).
    punct_translation = str.maketrans('', '', punctuation)
    stoplist = set(stopwords.words('english'))

    # ps = PorterStemmer()
    lem = WordNetLemmatizer()

    with open(filepath, 'r', encoding='latin-1') as f:
        reader = csv.reader(f, delimiter=",", )

        # Skip the header row
        next(reader, None)

        # Items per row:
        #   0 -> review id
        #   1 -> rating between 1-5
        #   2 -> year and month
        #   3 -> location of reviewer
        #   4 -> review text
        #   5 -> Disneyland location
        for row in reader:
            rating = int(row[1])

            if rating < 3:
                rating_label = 'negative'
            elif rating == 3:
                rating_label = 'indifferent'
            else:
                rating_label = 'positive'

            # review_text = row[4] \
            #     .translate(punct_translation) \
            #     .lower() \
            #     .strip()

            # tokenized = []
            # for token in word_tokenize(review_text):
            #     lemmatized = lem.lemmatize(token)
            #     if lemmatized not in stoplist:
            #         tokenized.append(lemmatized)

            # tokenized = [
                # Wordstemming -> SVM accuracy 0.815284 &  KNN accuracy 0.783638
                # ps.stem(token) for token in word_tokenize(review_text)

                # LEMMATIZATION -> SVM accuracy 0.810830 &  KNN accuracy 0.786217
                #  for token in word_tokenize(review_text)

                # token for token in word_tokenize(review_text)

            #     if token not in stoplist
            # ]

            # bag_of_words = ({t: True for t in tokenized}, rating_label)

            year, month = None, None
            if row[2] != 'missing':
                # 2019-4 for example
                [year, month] = row[2].split('-')

            dataset.append(
                {
                    # 'bag_of_words': bag_of_words,
                    # 'review_text': row[4],
                    # 'tokenized': tokenized,
                    'rating_label': rating_label,
                    'year': year,
                    'month': month,
                    'reviewer_location': row[3],
                    'disneyland_location': row[5].replace('Disneyland_', ''),
                    # 'doc_vector': nlp(' '.join(tokenized)).vector
                }
            )

    with open('cache.pickle', 'wb') as p:
        pickle.dump(dataset, p)

    return dataset


In [34]:
def split_train_test(feats, split=0.8):
    ''' Creates test, train and dev splits from the dataset ''' 
    random.Random(1).shuffle(feats)

    cutoff = int(len(feats) * split)
    tenpercent = int((len(feats) - cutoff) / 2)
    split = cutoff + tenpercent

    train_feats = feats[:cutoff]
    test_feats = feats[cutoff:split]
    dev_feats = feats[split:]

    print("  Training set: %i" % len(train_feats))
    print("  Test set: %i" % len(test_feats))
    print("  Development set: %i" % len(dev_feats))

    return train_feats, test_feats, dev_feats
dataset = create_dataset('../data/DisneylandReviews.csv')

train_feats, test_feats, dev_feats = split_train_test(dataset)

Reading in files...
  Training set: 34124
  Test set: 4266
  Development set: 4266


In [35]:
def evaluation_sklearn_model(model, test_examples, test_labels):
    y_pred = model.predict(test_examples)
    prec = precision_score(test_labels, y_pred, average='macro')
    rec = recall_score(test_labels, y_pred, average='macro')
    acc = accuracy_score(test_labels, y_pred)
    f = f1_score(test_labels, y_pred, average='macro')

    print(f"Accuracy: {acc}")
    print(f"Precision: {prec}\nRecall: {rec}\nF-score: {f}")
    print("Confusion matrix:")
    print(confusion_matrix(test_labels, y_pred))

In [36]:
reviewer_location_lookup = {}
i = 1
for item in dataset:
    if item['reviewer_location'] in reviewer_location_lookup:
        continue
    reviewer_location_lookup[item['reviewer_location']] = i
    i += 1

In [37]:
disney_loc_lookup = {}
i = 1
for item in dataset:
    if item['disneyland_location'] in disney_loc_lookup:
        continue
    disney_loc_lookup[item['disneyland_location']] = i
    i += 1

In [38]:
import numpy as np

only_reviewer_country_train = [[
    reviewer_location_lookup[i['reviewer_location']],
    disney_loc_lookup[i['disneyland_location']],
    int(i['year']),
    int(i['month']),
] for i in train_feats if i['year'] and i['month']]
only_reviewer_label_train = [i['rating_label'] for i in train_feats if i['year'] and i['month']]

only_reviewer_country_test = [[
    reviewer_location_lookup[i['reviewer_location']],
    disney_loc_lookup[i['disneyland_location']],
    int(i['year']),
    int(i['month']),
] for i in test_feats if i['year'] and i['month']]
only_reviewer_label_test = [i['rating_label'] for i in test_feats if i['year'] and i['month']]

print('\n-- BASELINE --\n')
baseline = DummyClassifier(strategy='most_frequent')
baseline.fit(only_reviewer_country_train, only_reviewer_label_train)
evaluation_sklearn_model(baseline, only_reviewer_country_test, only_reviewer_label_test)

print('\n-- SVM --\n')
svm = LinearSVC()
svm.fit(only_reviewer_country_train, only_reviewer_label_train)
evaluation_sklearn_model(svm, only_reviewer_country_test, only_reviewer_label_test)

print('\n-- KNN --\n')
knn = KNeighborsClassifier()
knn.fit(only_reviewer_country_train, only_reviewer_label_train)
evaluation_sklearn_model(knn, only_reviewer_country_test, only_reviewer_label_test)

print('\n-- Decision Tree --\n')
dtc = DecisionTreeClassifier()
dtc.fit(only_reviewer_country_train, only_reviewer_label_train)
evaluation_sklearn_model(dtc, only_reviewer_country_test, only_reviewer_label_test)


-- BASELINE --

Accuracy: 0.7887076537013802
Precision: 0.2629025512337934
Recall: 0.3333333333333333
F-score: 0.2939580995136551
Confusion matrix:
[[   0    0  485]
 [   0    0  357]
 [   0    0 3143]]

-- SVM --

/home/wessel/miniconda3/envs/mlp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/wessel/miniconda3/envs/mlp/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/wessel/miniconda3/envs/mlp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param